# Install and Import all the necessary packages and libraries

In [1]:
pip install paretoset

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist, mnist, cifar10, cifar100
import numpy as np
import time
import math
import copy
import random
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras import backend as K
import matplotlib.pyplot as plt
import pandas as pd
import gc
import multiprocessing
from multiprocessing import Process, Queue
from paretoset import paretoset

# Definition of programmer's parameters (Controller's block)

In [3]:
#@markdown ##Dataset
topic_prefix = "cifar10" # @param ["cifar10", "cifar100", "mnist", "fashion_mnist"] {type:"string"}
if topic_prefix == "cifar10":
  dataset = cifar10.load_data()
elif topic_prefix == "cifar100":
  dataset = cifar100.load_data()
elif topic_prefix == "mnist":
  dataset = mnist.load_data()
elif topic_prefix == "fashion_mnist":
  dataset = fashion_mnist.load_data()

#@markdown ##Synopses-based Training Optimization Configuration
# Percentage of the dataset that will be used for training
sample_size_low = 0.05 #@param {type:"slider", min:0, max:0.99, step:0.05}
sample_size_step = 0.1 #@param {type:"number"}
sample_size_high = 1 #@param {type:"slider", min:0, max:1, step:0.05}

# Percentage of the dataset that will be used for testing
perc_test = 1 #@param {type:"number"}

# The ID of the method that will be employed during sampling
# 0: Simple reservoir sampling in our initial training dataset
# 1: Reservoir sampling in each class based on the number of samples (per class)
sampling_method_id = 1 #@param {type:"integer"}

#@markdown ##NN Architecture Configuration

total_num_of_layers = 5 #@param {type:"integer"}

set_of_layers = ['conv', 'pool', 'dense'] #, 'drop']

# Number of epochs for the training process
# One Epoch is when an ENTIRE (training) dataset is passed forward and backward through the neural network only once.
# NOTE: An epoch is comprised of one or more batches.
num_of_epochs_low = 1 #@param {type:"slider", min:1, max:30, step:1}
num_of_epochs_step = 4 #@param {type:"integer"}
num_of_epochs_high = 20 #@param {type:"slider", min:1, max:30, step:1}

# The value for the learning rate for the training process (it is inserted to the Adam optimizer)
# During the optimization, the algorithm needs to take a series of tiny steps to descend the error mountain in order to minimize the error.
# The direction of the step is determined by the gradient, while the step size is determined by the learning rate.
lr_low = 1e-3 #@param {type:"number"}
lr_high = 1e-3 #@param {type:"number"}
lr_list = np.geomspace(lr_low, lr_high, num = int(np.log10(lr_high) - np.log10(lr_low)) + 1).tolist()

# The size of the batch for the training process
# Total number of training examples present in a single batch.
# or, number of samples processed before the model is updated.
size_of_batch_low = 64 #@param {type:"integer"}
size_of_batch_step = 16 #@param {type:"integer"}
size_of_batch_high = 64 #@param {type:"integer"}

#@markdown ##Tradeoff score Configuration
# Maximum training time above which we cancel the experiment (in seconds)
# It is used in the score (accuracy--speed tradeoff) formula
theta_parameter = 10 #@param {type:"number"}

# Weight for the accuracy of the model. Max value: 0.99
# There is also the weight of training speed of the model (1 - lamda_acc)
# It is used in the score (accuracy--speed tradeoff) formula
lamda_acc = 0.5 #@param {type:"number"}


#@markdown ##Other Configuration(s)

CONV_NEURONS_CONST = 32 #@param {type:"integer"}
CONV_NEURONS_BOUND = 256 #@param {type:"integer"}
DENSE_NEURONS_CONST = 128 #@param {type:"integer"}
DENSE_NEURONS_BOUND = 32 #@param {type:"integer"}

# Number of samples-architectures that pass generation to generation
g2g_promotions = 6 #@param {type:"integer"}

# Percentage of architectures that pass identical to the next generation (no expansion is performed)
g2g_percentage = 0.4 #@param {type:"number"}

mutation_lr_low = 0.0001
mutation_lr_high = 0.01

mutation_batch_low = 16
mutation_batch_high = 64

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step


# Load our image dataset

In [4]:
(train_images_all, train_labels_all), (test_images_all, test_labels_all) = dataset

train_images_all = np.vstack((train_images_all, train_images_all))
train_labels_all = np.concatenate((train_labels_all, train_labels_all))

# Get unique labels in our training dataset
unique_class_labels = np.unique(train_labels_all)

# Normalize pixel values in the trainning and the test datasets to be between 0 and 1
# TODO: parameter of normalization minmax scalar
train_images_all, test_images_all = train_images_all / 255.0, test_images_all / 255.0

if len(train_images_all.shape) == 3:
  # Make sure images have shape (28, 28, 1)
  train_images_all = np.expand_dims(train_images_all, -1)
  test_images_all = np.expand_dims(test_images_all, -1)
  print("x_train shape:", train_images_all.shape)
  print(train_images_all.shape[0], "train samples")
  print(test_images_all.shape[0], "test samples")

# Synopsis block of code

Optional: Check that our sampling method works appropriately

In [5]:
# A function that prints the occurence of each class in a list
def print_times_per_label(lst, labels_all):
  # Get unique labels in our training dataset
  unique_labels = np.unique(labels_all)
  for i in range(0, len(unique_labels)):
    print("Class", unique_labels[i], "has", lst.count(i), "samples in our dataset...")

**Reservoir Sampling Function**:
*Randomized algorithms for randomly choosing k samples from a list of n items, where n is either a very large or unknown number. Typically n is large enough that the list doesn’t fit into main memory.*

Algorithm implemented:


1.   Create an array reservoir[0,...,k-1] and copy first k items of stream[ ] to it.
2.   Now one by one consider all items from (k+1)th item to nth item.


> Steps

*   Generate a random number, denoted as j, from 0 to i where i is the index of the current item in stream[].
*   If j is in range 0 to k-1, replace reservoir[j] with stream[i]

In [6]:
# Select k items from a stream of items-data
import random

# A function to randomly select k items from stream[0..n-1].
def reservoir_sampling(stream, n, k):
  i = 0     # index for elements in stream[]

  # reservoir[] is the output array.
  # Initialize it with first k elements from stream[]
  reservoir = [0] * k

  for i in range(k):
    reservoir[i] = stream[i]

  # Iterate from the (k+1)th element to Nth element
  while(i < n):
    # Pick a random index from 0 to i.
    j = random.randrange(i+1)

    # If the randomly picked
    # index is smaller than k,
    # then replace the element
    # present at the index
    # with new element from stream
    if(j < k):
      reservoir[j] = stream[i]
    i+=1

  return reservoir

In [7]:
# A function that finds the size of each reservoir for every class depending on its occurence in the initial dataset
# and returns the unique labels that exist in our dataset along with the corresponding percentage
def reservoir_size_per_class(init_labels):

  # Get unique labels and their counts (how many times they appear) in our training dataset
  unique_labels, counts = np.unique(init_labels, return_counts = True)

  # Transform to list
  unique_labels_lst = unique_labels.tolist()
  counts_lst = counts.tolist()

  perc_per_class = []
  for i in range(len(unique_labels_lst)):
    perc_per_class.append(counts_lst[i]/len(init_labels))

  # print(perc_per_class)

  return perc_per_class, unique_labels_lst

"Pre-Processing Part (or Filtering)": Call the sampling mehod. Get samples from the training and the testing datasets.

In [8]:
def sampling_method(sampling_method_id, train_images_all, train_labels_all, sample_size, test_images_all, test_labels_all, perc_test):
  print("Percentage of filtering in our training dataset was set:")
  print(sample_size)
  if sampling_method_id == 0:
    # Simple reservoir sampling over the whole training dataset
    # Total size of the stream (or training dataset)
    n_train = len(train_images_all)

    # Number of samples that will be drawn
    k_train = int(n_train * sample_size)

    # Use the indexes of dataset in order to decide which samples will be drawn
    idx_tmp_train_list = list(range(0, n_train))

    # Find the indexes in order to construct the dataset that will be used during the training process
    idx_train = reservoir_sampling(idx_tmp_train_list, n_train, k_train)
  else:
    # Reservoir sampling in each class based on the number of samples (per class) that exist in the initial dataset
    # Find the size of each reservoir for every class depending on its occurence in the initial training dataset
    class_perc, unique_ids = reservoir_size_per_class(train_labels_all)

    # Stores the indexes (from all classes) in order to construct the dataset that will be used during the training process
    idx_train = []

    # Run for every single class the reservoir sampling seperately
    for i in range(0, len(unique_ids)):
      # Find the locations of each sample belonging to our class of interest
      tmp = np.where(train_labels_all == unique_ids[i])
      idx_of_class = tmp[0].tolist()

      # Run the reservoir sampling for the class of interest
      sampled_idx_of_class = reservoir_sampling(idx_of_class, len(idx_of_class), int(len(train_images_all) * sample_size * class_perc[i]))

      # Store the (sampled) samples from this class
      for j in range(0, len(sampled_idx_of_class)):
        idx_train.append(sampled_idx_of_class[j])

  # Store the corresponding images and labels from training dataset based on the sampled indexes
  train_images_lst = []
  for i in idx_train:
    train_images_lst.append(train_images_all[i])

  train_labels_lst = []
  for i in idx_train:
    train_labels_lst.append(train_labels_all[i])

  # Check the occurence of each class in the final training dataset
  print_times_per_label(train_labels_lst, train_labels_all)

  # Total size of the stream (or testing dataset)
  n_test = len(test_images_all)

  # Number of samples that will be drawn
  k_test = int(n_test * perc_test)

  # Use the indexes of dataset in order to decide which samples will be drawn
  idx_tmp_test_list = list(range(0, n_test))

  # Find the indexes in order to construct the dataset that will be used during the testing process
  idx_test = reservoir_sampling(idx_tmp_test_list, n_test, k_test)

  # Store the corresponding images and labels from testing dataset based on the sampled indexes
  test_images_lst = []
  for i in idx_test:
    test_images_lst.append(test_images_all[i])

  test_labels_lst = []
  for i in idx_test:
    test_labels_lst.append(test_labels_all[i])

  # Tranfsorm the lists that we stored our samples into arrays
  train_images = np.asarray(train_images_lst)
  train_labels = np.asarray(train_labels_lst)
  test_images = np.asarray(test_images_lst)
  test_labels = np.asarray(test_labels_lst)

  # Verify that the desired filtering was performed in both datasets
  print("Training dataset before sampling:")
  print(train_images_all.shape)
  print(train_labels_all.shape)
  print("Training dataset after sampling:")
  print(train_images.shape)
  print(train_labels.shape)

  print("Testing dataset before sampling:")
  print(test_images_all.shape)
  print(test_labels_all.shape)
  print("Testing dataset after sampling:")
  print(test_images.shape)
  print(test_labels.shape)

  return train_images, train_labels, test_images, test_labels

# CNN's architecture builder

In [9]:
def CrowdingDist(df, objective_columns):
    """
    :param df: DataFrame containing samples with features and objectives
    :param objective_columns: List of column names representing the objective values
    :return: List of crowding distances for each sample

    The crowding-distance computation requires sorting the population according to each objective function value
    in ascending order of magnitude. Thereafter, for each objective function, the boundary solutions (solutions with
    smallest and largest function values) are assigned an infinite distance value. All other intermediate solutions
    are assigned a distance value equal to the absolute normalized difference in the function values of two adjacent solutions.
    """
    n_samples = len(df)
    print(n_samples)
    distances = [0.0] * n_samples
    # create keys for fitness values
    crowd = [(tuple(row), i) for i, row in df.iterrows()]

    for objective_column in objective_columns:
        # Get the column position for the specified objective column
        obj_col_position = df.columns.get_loc(objective_column)

        crowd.sort(key=lambda element: element[0][obj_col_position])

        # After sorting, boundary solutions are assigned Inf
        # distances[crowd[0][1]] = float("Inf")
        # distances[crowd[-1][1]] = float("inf")

        if crowd[-1][0][obj_col_position] == crowd[0][0][obj_col_position]:  # If objective values are the same, skip this loop
            continue
        # normalization (max - min) as Denominator
        norm = float(crowd[-1][0][obj_col_position] - crowd[0][0][obj_col_position])
        # calculate each individual's Crowding Distance of i-th objective
        # technique: shift the list and zip
        for prev, cur, next_ in zip(crowd[:-2], crowd[1:-1], crowd[2:]):
            distances[cur[1]] += (next_[0][obj_col_position] - prev[0][obj_col_position]) / norm  # sum up the distance of i-th individual along each of the objectives

    return distances

In [10]:
def my_evaluate_crosmut(q, train_images, train_labels, test_images, test_labels, child_e, child_lr, child_bs, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND, l_child):
  error_flag = -1

  try:
    child_model, *_ = recreate_model(l_child, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND)

    # Check if the last layer is a Dense layer
    last_layer = child_model.layers[-1]

    # Check if the last layer of the network is dense
    # If it is just add the last dense layer for the classification
    # Otherwise you should first add a flatten layer
    if isinstance(last_layer, tf.keras.layers.Dense):
        child_model.add(layers.Dense(len(unique_class_labels), activation='softmax'))
    else:
        child_model.add(layers.Flatten())
        child_model.add(layers.Dense(len(unique_class_labels), activation='softmax'))

  except ValueError:
      print("No valid input...:(")
      error_flag = 1

  if error_flag == -1:
    child_model.compile(optimizer=Adam(child_lr), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    child_model.summary()

    start = time.time()

    blackbox = child_model.fit(x=train_images,
                        y=train_labels,
                        epochs=child_e,
                        batch_size=child_bs
                        )
    stop = time.time()

    # Compute the training speed of this CNN architecture
    tr_time = stop - start

    # Compute the accuracy of our training model in the testing dataset
    test_loss, test_acc = child_model.evaluate(test_images,  test_labels, verbose=2)

    # Compute the metric that captures the accuracy--speed tradeoff
    # Original formula
    # tradeOff_metric = lamda_acc * test_acc - (1 - lamda_acc) * math.tanh(tr_time/theta_parameter - 1)

    # Linear formula
    # tradeOff_metric = lamda_acc * test_acc + (1 - lamda_acc) * (1 - min(1, tr_time/theta_parameter))

    # Multiplicative formula
    # tradeOff_metric = test_acc ** lamda_acc * (1 - min(1, tr_time/theta_parameter)) ** (1 - lamda_acc)

    # Harmonic mean
    # tradeOff_metric = (2 * test_acc * (1 - min(1, tr_time/theta_parameter))) / (test_acc + 1 - min(1, tr_time/theta_parameter) + 1e-5)

    # Softmin
    #tradeOff_metric = 1 - np.log(1 + np.exp(-0.1 * test_acc) + np.exp(0.1 * min(1, tr_time/theta_parameter)))

    # Sigmoid
    # tradeOff_metric = (1 / (1 + np.exp(-test_acc))) * (1 - 1 / (1 + np.exp(-min(1, tr_time/theta_parameter))))

    # Piecewise
    # tradeOff_metric = test_acc if tr_time <= theta_parameter else test_acc * max(0, 1 - 0.5 * (tr_time - theta_parameter))

    # F1 like
    # alpha = 1
    # tradeOff_metric = ((1 + alpha**2) * test_acc * min(1, tr_time/theta_parameter)) / (alpha ** 2 * test_acc + min(1, tr_time/theta_parameter))

    # Exp decay penalty
    tradeOff_metric = test_acc * np.exp(-0.5 * min(1, tr_time/theta_parameter))

    # inverse additive penalty
    # tradeOff_metric = test_acc / (1 + 0.5 * min(1, tr_time/theta_parameter))

    # min-based tradeoff
    # tradeOff_metric = min(test_acc, min(1, tr_time/theta_parameter))

    # product with complement
    # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter))

    # square root compensation
    # tradeOff_metric = np.sqrt(test_acc) * np.sqrt(1 - min(1, tr_time/theta_parameter))

    # jacard style normalization
    # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter)) / (test_acc + (1 - min(1, tr_time/theta_parameter)) - test_acc * (1 - min(1, tr_time/theta_parameter)))

    # log-sigmoid dominance
    # tradeOff_metric = 1 / (1 + np.exp(-0.5 * (test_acc - (1 - min(1, tr_time/theta_parameter)))))

    del child_model

    print("========================== EDW EINAI TO MULTI333333.......")
    print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
    print(f"Training time: ", tr_time)
    print(tradeOff_metric)

    q.put([test_acc, tr_time, tradeOff_metric])

  else:
    q.put([0, 1000000000, 0])


In [11]:
def CrossoverMutationResult(df):

  # The first g2g_promotions * g2g_percentage NN architectures will continue to the next generation...
  result = df.head(math.floor(g2g_promotions * g2g_percentage)).copy(deep = True)

  # Create an empty DataFrame with specified column names
  column_names = ['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score']
  tmp_df = pd.DataFrame(columns=column_names)

  ##############################################################
  # Crossover and mutation: The first with the remaining g2g_promotions * g2g_percentage architectures, e.g., 1 + 2, 1 + 3, 1 + 4,...
  ##############################################################
  idx = 0
  for i in range(1, (math.floor(g2g_promotions * g2g_percentage))):
    # Get the features of each 'parent', i.e, layers, sampling rates and epochs
    print()
    l1 = df['LayerType'].iloc[idx]
    sr1 = df['SamplingRate'].iloc[idx]
    e1 = df['Epochs'].iloc[idx]
    lr1 = df['LearningRate'].iloc[idx]
    bs1 = df['BatchSize'].iloc[idx]

    l2 = df['LayerType'].iloc[idx + i]
    sr2 = df['SamplingRate'].iloc[idx + i]
    e2 = df['Epochs'].iloc[idx + i]
    lr2 = df['LearningRate'].iloc[idx + i]
    bs2 = df['BatchSize'].iloc[idx + i]

    #  -----------------------------  Crossover  -----------------------------
    # The 'length' of the resulted child will be equal to the min length of its parents
    # For each layer get randomly (50% chance) the corresponding layer of one of its parents
    # Moreover, from one parent get its sampling rate and from the other the number of epochs
    l_min = min(len(l1), len(l2))
    print(l_min)

    l_child = []
    for l in range(0, l_min):

      if l > 0:
        if l_child[-1] == 'dense':
          l_child.append('dense')
          continue

      select_parent = random.randint(0, 1)
      if select_parent == 0:
        l_child.append(l1[l])
      else:
        l_child.append(l2[l])

    select_parent = random.randint(0, 5)
    if select_parent == 0:
      child_sr = sr1
      child_e = e1
      child_lr = lr2
      child_bs = bs2
    elif select_parent == 1:
      child_sr = sr2
      child_e = e2
      child_lr = lr1
      child_bs = bs1
    elif select_parent == 2:
      child_sr = sr1
      child_e = e2
      child_lr = lr1
      child_bs = bs2
    elif select_parent == 3:
      child_sr = sr2
      child_e = e1
      child_lr = lr2
      child_bs = bs1
    elif select_parent == 4:
      child_sr = sr1
      child_e = e2
      child_lr = lr2
      child_bs = bs1
    else:
      child_sr = sr2
      child_e = e1
      child_lr = lr1
      child_bs = bs2

    # -----------------------------  Mutation  -----------------------------
    # Perform a mutation in either the sampling rate or the number of epochs.
    select_feature = random.randint(0, 3)
    if select_feature == 0:
      print("Mutation on Sampling Rate")
      random_float = random.uniform(sample_size_low, sample_size_high)
      child_sr = random_float
    elif select_feature == 1:
      print("Mutation on Learning Rate")
      random_float = random.uniform(mutation_lr_low, mutation_lr_high)
      child_lr = random_float
    elif select_feature == 2:
      print("Mutation on batch size")
      random_int = random.randint(mutation_batch_low, mutation_batch_high)
      child_bs = random_int
    else:
      print("Mutation on Epochs")
      random_int = random.randint(num_of_epochs_low, num_of_epochs_high)
      child_e = random_int


    # Perform the sampling
    train_images, train_labels, test_images, test_labels = sampling_method(sampling_method_id, train_images_all, train_labels_all, child_sr, test_images_all, test_labels_all, perc_test)
    q = Queue()
    process_eval = multiprocessing.Process(target=my_evaluate_crosmut, args=(q, train_images, train_labels, test_images, test_labels, child_e, child_lr, child_bs, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND, l_child))
    process_eval.start()
    test_acc, tr_time, tradeOff_metric = q.get()
    process_eval.join()

    # Print the results.
    print()
    print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
    print(f"Training time: ", tr_time)
    print(tradeOff_metric)
    print()

    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    gc.collect()
    del train_images
    del train_labels
    del test_images
    del test_labels
    #tf.compat.v1.reset_default_graph()

    # Create a dictionary representing a row to add
    new_row_data = {
        'LayerType': l_child,
        'SamplingRate': child_sr,
        'LearningRate': child_lr,
        'BatchSize': child_bs,
        'Epochs': child_e,
        'Accuracy': test_acc,
        'TrainingSpeed': tr_time,
        'Score': tradeOff_metric
    }

    # Create a new DataFrame with the single row
    new_row_df = pd.DataFrame([new_row_data])

    # Concatenate the empty DataFrame and the new DataFrame
    tmp_df = pd.concat([tmp_df, new_row_df], ignore_index=True)

    print("-----------------> End of first phase crossovers/mutations")

  print(tmp_df)

  ##############################################################
  # Crossover and mutation topK with downK, e.g., 1 + 6, 2 + 7, 3 = 8, 4 + 9 and 5 + 10
  ##############################################################

  print(df)
  for i in range(math.floor(len(df) * 0.5)):
    print()
    l1 = df['LayerType'].iloc[i]
    sr1 = df['SamplingRate'].iloc[i]
    e1 = df['Epochs'].iloc[i]
    lr1 = df['LearningRate'].iloc[i]
    bs1 = df['BatchSize'].iloc[i]

    l2 = df['LayerType'].iloc[i + math.floor(len(df) * 0.5)]
    sr2 = df['SamplingRate'].iloc[i + math.floor(len(df) * 0.5)]
    e2 = df['Epochs'].iloc[i + math.floor(len(df) * 0.5)]
    lr2 = df['LearningRate'].iloc[i + math.floor(len(df) * 0.5)]
    bs2 = df['BatchSize'].iloc[i + math.floor(len(df) * 0.5)]

    # Crossover (similar to the first phase crossover)
    l_min = min(len(l1), len(l2))
    print(l_min)

    l_child = []
    for l in range(0, l_min):

      if l > 0:
        if l_child[-1] == 'dense':
          l_child.append('dense')
          continue

      select_parent = random.randint(0, 1)
      if select_parent == 0:
        l_child.append(l1[l])
      else:
        l_child.append(l2[l])

    select_parent = random.randint(0, 5)
    if select_parent == 0:
      child_sr = sr1
      child_e = e1
      child_lr = lr2
      child_bs = bs2
    elif select_parent == 1:
      child_sr = sr2
      child_e = e2
      child_lr = lr1
      child_bs = bs1
    elif select_parent == 2:
      child_sr = sr1
      child_e = e2
      child_lr = lr1
      child_bs = bs2
    elif select_parent == 3:
      child_sr = sr2
      child_e = e1
      child_lr = lr2
      child_bs = bs1
    elif select_parent == 4:
      child_sr = sr1
      child_e = e2
      child_lr = lr2
      child_bs = bs1
    else:
      child_sr = sr2
      child_e = e1
      child_lr = lr1
      child_bs = bs2

    # -----------------------------  Mutation  -----------------------------
    # Perform a mutation in either the sampling rate or the number of epochs.
    select_feature = random.randint(0, 3)
    if select_feature == 0:
      print("Mutation on Sampling Rate")
      #random_float = random.uniform(-0.2, 0.2)
      #child_sr = child_sr + random_float
      random_float = random.uniform(sample_size_low, sample_size_high)
      child_sr = random_float
    elif select_feature == 1:
      print("Mutation on Learning Rate")
      random_float = random.uniform(mutation_lr_low, mutation_lr_high)
      child_lr = random_float
    elif select_feature == 2:
      print("Mutation on batch size")
      random_int = random.randint(mutation_batch_low, mutation_batch_high)
      child_bs = random_int
    else:
      print("Mutation on Epochs")
      # random_int = random.randint(-5, 5)
      # child_e = child_e + random_int
      random_int = random.randint(num_of_epochs_low, num_of_epochs_high)
      child_e = random_int


    # Perform the sampling
    train_images, train_labels, test_images, test_labels = sampling_method(sampling_method_id, train_images_all, train_labels_all, child_sr, test_images_all, test_labels_all, perc_test)
    q = Queue()
    process_eval = multiprocessing.Process(target=my_evaluate_crosmut, args=(q, train_images, train_labels, test_images, test_labels, child_e, child_lr, child_bs, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND, l_child))
    process_eval.start()
    test_acc, tr_time, tradeOff_metric = q.get()
    process_eval.join()

    # Print the results.
    print()
    print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
    print(f"Training time: ", tr_time)
    print(tradeOff_metric)
    print()

    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    gc.collect()
    del train_images
    del train_labels
    del test_images
    del test_labels
    #tf.compat.v1.reset_default_graph()

    # Create a dictionary representing a row to add
    new_row_data = {
        'LayerType': l_child,
        'SamplingRate': child_sr,
        'LearningRate': child_lr,
        'BatchSize': child_bs,
        'Epochs': child_e,
        'Accuracy': test_acc,
        'TrainingSpeed': tr_time,
        'Score': tradeOff_metric
    }

    # Create a new DataFrame with the single row
    new_row_df = pd.DataFrame([new_row_data])

    # Concatenate the empty DataFrame and the new DataFrame
    tmp_df = pd.concat([tmp_df, new_row_df], ignore_index=True)

    print("-----------------> End of second phase crossovers/mutations")

  print(tmp_df)

  ###################################################
  # Add the original networks (positions that are after the first g2g_percentage), e.g. if g2g_rercentage is 0.2 and g2g_promotion = 10, then you will add the architectures in positions 3, 4, ..., 10
  ###################################################

  tmp_df = pd.concat([tmp_df, df_first.iloc[math.floor(g2g_promotions * g2g_percentage):]])
  print("-----------------> Add the original networks (positions after the first g2g_percentage)")
  print(tmp_df)

  tmp_df = tmp_df.drop(columns='CD')
  tmp_df = tmp_df.reset_index(drop=True)

  print(tmp_df)

  ###################################################
  # Compute Crowding Distance
  ###################################################

  objective_columns = ['Accuracy', 'TrainingSpeed']
  crowding_distances = CrowdingDist(tmp_df, objective_columns)

  tmp_df['CD'] = crowding_distances
  tmp_df.sort_values('CD', ascending = False, inplace = True)

  tmp_df.reset_index(drop = True, inplace = True)

  # Get the (1 - g2g_percentage) architectures with the highest Crowding Distance
  df_cd_crosmut = tmp_df.head(math.ceil(g2g_promotions * (1 - g2g_percentage)))


  ###################################################
  # Create the final result
  ###################################################
  result = pd.concat([result, df_cd_crosmut])
  result = result.drop(columns='CD')
  result.reset_index(drop = True, inplace = True)

  return result

In [12]:
def recreate_model(layers_lst, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND):
  # Initialize a sequential model
  model = models.Sequential()

  # Define the number of neurons for conv and dense layers
  conv_tmp2 = CONV_NEURONS_CONST
  dense_tmp2 = DENSE_NEURONS_CONST

  # Recreate the model
  for count, layer in enumerate(layers_lst):
    # First layer has to be a convolutional one
    if layer == 'conv' and count == 0:
      model.add(layers.Conv2D(int(conv_tmp2), (3, 3), activation='relu', input_shape = dataset_shape))
      conv_tmp2 = conv_tmp2 * 2
    # For the other layers
    else:
      if layer == 'conv':
        # Add a conv layer by doubling its neurons if they do not violate our user-defined bound
        if conv_tmp2 <= CONV_NEURONS_BOUND:
          model.add(layers.Conv2D(int(conv_tmp2), (3, 3), activation='relu'))
          conv_tmp2 = conv_tmp2 * 2
        else:
          model.add(layers.Conv2D(int(CONV_NEURONS_BOUND), (3, 3), activation='relu'))
          conv_tmp2 = CONV_NEURONS_BOUND
      elif layer == 'pool':
        # Add a pool layer
        model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))
      elif layer == 'drop':  # Dropout
        model.add(layers.Dropout(rate=0.5))
      else:
        # If the next to-be-added-layer is dense and no other dense layer has been added so far, then add a flatten layer first...
        if dense_tmp2 == DENSE_NEURONS_CONST:
          model.add(layers.Flatten())
        # Add a dense layer by reducing (* 0.5) its neurons if they do not violate our user-defined bound
        if dense_tmp2 >= DENSE_NEURONS_BOUND:
          model.add(layers.Dense(int(dense_tmp2), activation='relu'))
          dense_tmp2 = dense_tmp2 / 2
        else:
          model.add(layers.Dense(int(DENSE_NEURONS_BOUND), activation='relu'))
          dense_tmp2 = DENSE_NEURONS_BOUND

  return model, conv_tmp2, dense_tmp2

In [13]:
def my_evaluate_first_phase(q, train_images, train_labels, test_images, test_labels, epochs, lr, size_of_batch):
  # Initialize a sequential network
  model = models.Sequential()

  # Define the neurons of conv and dense layers based on user's input
  conv_tmp = CONV_NEURONS_CONST
  dense_tmp = DENSE_NEURONS_CONST

  # Add the (first) conv layer
  model.add(layers.Conv2D(int(CONV_NEURONS_CONST), (3, 3), activation='relu', input_shape = dataset_shape))
  conv_tmp = conv_tmp * 2

  # Add manually a flatten and a dense layer in order to evaluate the network
  model.add(layers.Flatten())
  model.add(layers.Dense(len(unique_class_labels), activation='softmax'))

  model.compile(optimizer=Adam(lr), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
  model.summary()

  start = time.time()

  blackbox = model.fit(x=train_images,
                      y=train_labels,
                      epochs=epochs,
                      batch_size=size_of_batch
                      )
  stop = time.time()

  # Compute the training speed of this CNN architecture
  tr_time = stop - start

  # Compute the accuracy of our training model in the testing dataset
  test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

  # Compute the metric that captures the accuracy--speed tradeoff
  # Original formula
  # tradeOff_metric = lamda_acc * test_acc - (1 - lamda_acc) * math.tanh(tr_time/theta_parameter - 1)

  # Linear formula
  # tradeOff_metric = lamda_acc * test_acc + (1 - lamda_acc) * (1 - min(1, tr_time/theta_parameter))

  # Multiplicative formula
  # tradeOff_metric = test_acc ** lamda_acc * (1 - min(1, tr_time/theta_parameter)) ** (1 - lamda_acc)

  # Harmonic mean
  # tradeOff_metric = (2 * test_acc * (1 - min(1, tr_time/theta_parameter))) / (test_acc + 1 - min(1, tr_time/theta_parameter) + 1e-5)

  # Softmin
  # tradeOff_metric = 1 - np.log(1 + np.exp(-0.1 * test_acc) + np.exp(0.1 * min(1, tr_time/theta_parameter)))

  # Sigmoid
  # tradeOff_metric = (1 / (1 + np.exp(-test_acc))) * (1 - 1 / (1 + np.exp(-min(1, tr_time/theta_parameter))))

  # Piecewise
  # tradeOff_metric = test_acc if tr_time <= theta_parameter else test_acc * max(0, 1 - 0.5 * (tr_time - theta_parameter))

  # F1 like
  # alpha = 1
  # tradeOff_metric = ((1 + alpha**2) * test_acc * min(1, tr_time/theta_parameter)) / (alpha ** 2 * test_acc + min(1, tr_time/theta_parameter))

  # Exp decay penalty
  tradeOff_metric = test_acc * np.exp(-0.5 * min(1, tr_time/theta_parameter))

  # inverse additive penalty
  # tradeOff_metric = test_acc / (1 + 0.5 * min(1, tr_time/theta_parameter))

  # min-based tradeoff
  # tradeOff_metric = min(test_acc, min(1, tr_time/theta_parameter))

  # product with complement
  # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter))

  # square root compensation
  # tradeOff_metric = np.sqrt(test_acc) * np.sqrt(1 - min(1, tr_time/theta_parameter))

  # jacard style normalization
  # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter)) / (test_acc + (1 - min(1, tr_time/theta_parameter)) - test_acc * (1 - min(1, tr_time/theta_parameter)))

  # log-sigmoid dominance
  # tradeOff_metric = 1 / (1 + np.exp(-0.5 * (test_acc - (1 - min(1, tr_time/theta_parameter)))))

  # Delete the Keras model with these hyper-parameters from memory.
  del model

  print("========================== EDW EINAI TO MULTI1111111111111111.......")
  print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
  print(f"Training time: ", tr_time)
  print(tradeOff_metric)

  q.put([test_acc, tr_time, tradeOff_metric])


In [14]:
def my_evaluate_rest_phase(q, train_images, train_labels, test_images, test_labels, layer, current_df, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND):
  error_flag = -1

  # Recreate the network that consist of the best layers that we found in each of the previous steps/iterations
  model, conv_tmp, dense_tmp = recreate_model(current_df['LayerType'], dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND)

  try:
    # If the to-be-added-layer is conv
    if layer == 'conv':
      # Add a conv layer by doubling its neurons if they do not violate our user-defined bound
      if conv_tmp <= CONV_NEURONS_BOUND:
        model.add(layers.Conv2D(int(conv_tmp), (3, 3), activation='relu'))
        conv_tmp = conv_tmp * 2
      else:
        model.add(layers.Conv2D(int(CONV_NEURONS_BOUND), (3, 3), activation='relu'))
    # If the to-be-added-layer is pool
    elif layer == 'pool':
      model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))
    elif layer == 'drop':  # Dropout
        model.add(layers.Dropout(rate=0.5))
    # If the to-be-added-layer is dense
    else:
      # If the next to-be-added-layer is dense and no other dense layer has been added so far, then add a flatten layer first...
      if not isinstance(model.layers[-1], tf.keras.layers.Dense):
        model.add(layers.Flatten())
      # Add a dense layer by reducing (* 0.5) its neurons if they do not violate our user-defined bound
      if dense_tmp >= DENSE_NEURONS_BOUND:
        model.add(layers.Dense(int(dense_tmp), activation='relu'))
        dense_tmp = dense_tmp / 2
      else:
        model.add(layers.Dense(int(DENSE_NEURONS_BOUND), activation='relu'))


    # Check if the last layer is a Dense layer
    last_layer = model.layers[-1]

    # Check if the last layer of the network is dense
    # If it is just add the last dense layer for the classification
    # Otherwise you should first add a flatten layer
    if isinstance(last_layer, tf.keras.layers.Dense):
        model.add(layers.Dense(len(unique_class_labels), activation='softmax'))
    else:
        model.add(layers.Flatten())
        model.add(layers.Dense(len(unique_class_labels), activation='softmax'))
  except ValueError:
    print("No valid input...:(")
    error_flag = 1


  if error_flag == -1:
    model.compile(optimizer=Adam(current_df['LearningRate']), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    model.summary()

    start = time.time()

    blackbox = model.fit(x=train_images,
                        y=train_labels,
                        epochs=current_df['Epochs'],
                        batch_size=current_df['BatchSize']
                        )
    stop = time.time()

    # Compute the training speed of this CNN architecture
    tr_time = stop - start

    # Compute the accuracy of our training model in the testing dataset
    test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

    # Compute the metric that captures the accuracy--speed tradeoff
    # Original formula
    # tradeOff_metric = lamda_acc * test_acc - (1 - lamda_acc) * math.tanh(tr_time/theta_parameter - 1)

    # Linear formula
    # tradeOff_metric = lamda_acc * test_acc + (1 - lamda_acc) * (1 - min(1, tr_time/theta_parameter))

    # Multiplicative formula
    # tradeOff_metric = test_acc ** lamda_acc * (1 - min(1, tr_time/theta_parameter)) ** (1 - lamda_acc)

    # Harmonic mean
    # tradeOff_metric = (2 * test_acc * (1 - min(1, tr_time/theta_parameter))) / (test_acc + 1 - min(1, tr_time/theta_parameter) + 1e-5)

    # Softmin
    #tradeOff_metric = 1 - np.log(1 + np.exp(-0.1 * test_acc) + np.exp(0.1 * min(1, tr_time/theta_parameter)))

    # Sigmoid
    # tradeOff_metric = (1 / (1 + np.exp(-test_acc))) * (1 - 1 / (1 + np.exp(-min(1, tr_time/theta_parameter))))

    # Piecewise
    # tradeOff_metric = test_acc if tr_time <= theta_parameter else test_acc * max(0, 1 - 0.5 * (tr_time - theta_parameter))

    # F1 like
    # alpha = 1
    # tradeOff_metric = ((1 + alpha**2) * test_acc * min(1, tr_time/theta_parameter)) / (alpha ** 2 * test_acc + min(1, tr_time/theta_parameter))

    # Exp decay penalty
    tradeOff_metric = test_acc * np.exp(-0.5 * min(1, tr_time/theta_parameter))

    # inverse additive penalty
    # tradeOff_metric = test_acc / (1 + 0.5 * min(1, tr_time/theta_parameter))

    # min-based tradeoff
    # tradeOff_metric = min(test_acc, min(1, tr_time/theta_parameter))

    # product with complement
    # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter))

    # square root compensation
    # tradeOff_metric = np.sqrt(test_acc) * np.sqrt(1 - min(1, tr_time/theta_parameter))

    # jacard style normalization
    # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter)) / (test_acc + (1 - min(1, tr_time/theta_parameter)) - test_acc * (1 - min(1, tr_time/theta_parameter)))

    # log-sigmoid dominance
    # tradeOff_metric = 1 / (1 + np.exp(-0.5 * (test_acc - (1 - min(1, tr_time/theta_parameter)))))

    del model

    print("========================== EDW EINAI TO MULTI2222222222.......")
    print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
    print(f"Training time: ", tr_time)
    print(tradeOff_metric)

    q.put([test_acc, tr_time, tradeOff_metric])
  else:
    q.put([0, 1000000000, 0])

In [15]:
start_program = time.time()

# Get the shape of the input dataset
dataset_shape = train_images_all.shape[1:]

# Store the best type of layer of each step
best_layers = []

# Loop for the number of layers that user indicated
for step in range(1, total_num_of_layers + 1):

  # If this is the first step/iteration
  if step == 1:

    # The first layer in this set of datasets has to be a convolutional one!
    # So our "search space" is all combinations of epochs and sampling rates based on user's input
    for sample_size in np.arange(sample_size_low, sample_size_high + 0.01, sample_size_step):
      for epochs in range(num_of_epochs_low, num_of_epochs_high + 1, num_of_epochs_step):
        for lr in lr_list:
          for size_of_batch in range(size_of_batch_low, size_of_batch_high + 1, size_of_batch_step):
            # Perform the sampling
            train_images, train_labels, test_images, test_labels = sampling_method(sampling_method_id, train_images_all, train_labels_all, sample_size, test_images_all, test_labels_all, perc_test)

            q = Queue()
            process_eval = multiprocessing.Process(target=my_evaluate_first_phase, args=(q, train_images, train_labels, test_images, test_labels, epochs, lr, size_of_batch))
            process_eval.start()
            test_acc, tr_time, tradeOff_metric = q.get()
            process_eval.join()

            # Print the results.
            print()
            print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
            print(f"Training time: ", tr_time)
            print(tradeOff_metric)
            print()

            # Delete the dfs.
            del train_images
            del train_labels
            del test_images
            del test_labels

            # Clear the Keras session, otherwise it will keep adding new
            # models to the same TensorFlow graph each time we create
            # a model with a different set of hyper-parameters.
            K.clear_session()
            tf.compat.v1.reset_default_graph()

            # If this is the first combination that is examined then create the dataframe and store the result
            if sample_size == sample_size_low and epochs == num_of_epochs_low and lr == lr_list[0] and size_of_batch == size_of_batch_low:
              df = pd.DataFrame([[['conv'], sample_size, lr, size_of_batch, epochs, test_acc, tr_time, tradeOff_metric]], columns=['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score'])
            # else append the result to the existing dataframe
            else:
              df_tmp = pd.DataFrame([[['conv'], sample_size, lr, size_of_batch, epochs, test_acc, tr_time, tradeOff_metric]], columns=['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score'])
              df = pd.concat([df, df_tmp], axis = 0, ignore_index=True)
              del df_tmp

  # If this is not the first step/iteration
  else:
    # This boolean variable indicates if this is the first examined combination in order to create a df
    flag = True

    # Get the df of the previous step/iteration
    df_tmp2 = best_layers[-1]

    # For every pareto optimal solution (of the previous step)
    for pareto_layer in range(0, len(df_tmp2)):   #math.floor(g2g_promotions * g2g_percentage)
      current_df = df_tmp2.iloc[pareto_layer]

      # For all the available types of layers
      for layer in set_of_layers:

        # After a dense laye you can only add a dense layer
        # So, don't examine the other types of layers
        if current_df['LayerType'][-1] == 'dense' and layer != 'dense':
          continue

        # Perform the sampling
        train_images, train_labels, test_images, test_labels = sampling_method(sampling_method_id, train_images_all, train_labels_all, current_df['SamplingRate'], test_images_all, test_labels_all, perc_test)

        q = Queue()
        process_eval = multiprocessing.Process(target=my_evaluate_rest_phase, args=(q, train_images, train_labels, test_images, test_labels, layer, current_df, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND))
        process_eval.start()
        test_acc, tr_time, tradeOff_metric = q.get()
        process_eval.join()

        # Print the results.
        print()
        print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
        print(f"Training time: ", tr_time)
        print(tradeOff_metric)
        print()

        # Clear the Keras session, otherwise it will keep adding new
        # models to the same TensorFlow graph each time we create
        # a model with a different set of hyper-parameters.
        K.clear_session()
        gc.collect()
        del train_images
        del train_labels
        del test_images
        del test_labels
        # tf.compat.v1.reset_default_graph()

        # If this is the first combination that is examined then create the dataframe and store the result
        if flag:
          flag = False
          so_far_layers = current_df['LayerType'].copy()
          so_far_layers.append(layer)
          df = pd.DataFrame([[so_far_layers, current_df['SamplingRate'], current_df['LearningRate'], current_df['BatchSize'], current_df['Epochs'], test_acc, tr_time, tradeOff_metric]], columns=['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score'])
        # else append the result to the existing df
        else:
          so_far_layers = current_df['LayerType'].copy()
          so_far_layers.append(layer)
          df_tmp = pd.DataFrame([[so_far_layers, current_df['SamplingRate'], current_df['LearningRate'], current_df['BatchSize'], current_df['Epochs'], test_acc, tr_time, tradeOff_metric]], columns=['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score'])
          df = pd.concat([df, df_tmp], axis = 0, ignore_index=True)
          del df_tmp


  print(df)

  # If this is after the first iteration-layer, then the best g2g_promotions * g2g_percentage are not expanded and as such we append them directly to the resulted df
  # Note that df_tmp2 is the dataframe that contains the best solutions from the previous iteration-layer
  # if step > 1:
  #   print(df_tmp2)
  #   df = pd.concat([df_tmp2.iloc[:math.floor(g2g_promotions * g2g_percentage)], df], ignore_index=True)
  #   if 'CD' in df.columns:
  #     df = df.drop('CD', axis=1)
  #   print(df)

  # flag that indicates if the to-be-computed rank solutions (pareto front) is the first one
  first_rank = True
  counter = 0

  while True:

    counter += 1

    print("######################")
    print("Our candidates")
    print("######################")
    print(df)

    # Keep the columns of the dataframe that we need to compute the Pareto opt, i.e., accuracy and training speed
    df2 = df.loc[:, df.columns.drop(['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Score'])]

    # Set the appropriate mask, i.e., max the accuracy, minimize the training speed
    mask = paretoset(df2, sense=["max", "min"])

    # Apply the mask and get the result
    paretoset_nn = df[mask]
    paretoset_nn.reset_index(drop = True, inplace = True)

    # If this is the first rank create the df, else just append the result to the existing df
    if first_rank:
      my_ranks = copy.deepcopy(paretoset_nn)
      first_rank = False
    else:
      my_ranks = pd.concat([my_ranks, paretoset_nn], ignore_index=True)

    print("######################")
    print(f"The {counter}-ranks that have qualified (previous ranks are also included if they exist...)")
    print("######################")
    print(my_ranks)

    # Check if you need the next rank
    if my_ranks.shape[0] >= g2g_promotions:
      print("----------------------> We do not need the next rank...")
      break

    else:
      #####################################################################
      #     Remove the ith rank in order to compute the ith + 1 rank
      #####################################################################
      df['LayerType'] = df['LayerType'].apply(tuple)
      paretoset_nn['LayerType'] = paretoset_nn['LayerType'].apply(tuple)

      result_df = pd.merge(df, paretoset_nn, on=['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score'], how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

      result_df['LayerType'] = result_df['LayerType'].apply(list)
      df['LayerType'] = df['LayerType'].apply(list)
      paretoset_nn['LayerType'] = paretoset_nn['LayerType'].apply(list)

      df = copy.deepcopy(result_df)

  # Compute the crowding distance between all the architectures that were qualified based on their rank
  objective_columns = ['Accuracy', 'TrainingSpeed']
  crowding_distances = CrowdingDist(my_ranks, objective_columns)

  my_ranks['CD'] = crowding_distances
  my_ranks.sort_values('CD', ascending = False, inplace = True)
  print(my_ranks)

  # Selecting the top g2g_promotions * g2g_percentage rows with the maximum 'f1'
  top_max_score = my_ranks.nlargest(math.floor(g2g_promotions * g2g_percentage), 'Score')

  my_ranks['LayerType'] = my_ranks['LayerType'].apply(tuple)
  top_max_score['LayerType'] = top_max_score['LayerType'].apply(tuple)

  # Adding the top g2g_promotions * g2g_percentage rows to the top of the original DataFrame and removing duplicates
  my_ranks = pd.concat([top_max_score, my_ranks]).drop_duplicates().reset_index(drop=True)

  my_ranks['LayerType'] = my_ranks['LayerType'].apply(list)
  top_max_score['LayerType'] = top_max_score['LayerType'].apply(list)

  my_ranks.reset_index(drop = True, inplace = True)
  print(my_ranks)

  # Qualify the best based on ther CD score
  df_first = my_ranks.head(g2g_promotions)
  print(df_first)

  # Crossover + Mutation function
  df_crossmut = CrossoverMutationResult(df_first)

  # Add the dataframe that contains the pareto opt architectures for this step/iteration and the corresponding metrics
  best_layers.append(df_crossmut)

Percentage of filtering in our training dataset was set:
0.05
Class 0 has 500 samples in our dataset...
Class 1 has 500 samples in our dataset...
Class 2 has 500 samples in our dataset...
Class 3 has 500 samples in our dataset...
Class 4 has 500 samples in our dataset...
Class 5 has 500 samples in our dataset...
Class 6 has 500 samples in our dataset...
Class 7 has 500 samples in our dataset...
Class 8 has 500 samples in our dataset...
Class 9 has 500 samples in our dataset...
Training dataset before sampling:
(100000, 32, 32, 3)
(100000, 1)
Training dataset after sampling:
(5000, 32, 32, 3)
(5000, 1)
Testing dataset before sampling:
(10000, 32, 32, 3)
(10000, 1)
Testing dataset after sampling:
(10000, 32, 32, 3)
(10000, 1)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 28800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │       288,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 288,906 (1.10 MB)

...

 Trainable params: 125,258 (489.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3


/usr/local/lib/python3.12/dist-packages/keras/src/backend/tensorflow/nn.py:717: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


592/592 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.2901 - loss: 1.9378
Epoch 2/3
592/592 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4789 - loss: 1.4630
Epoch 3/3
592/592 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5322 - loss: 1.3198
313/313 - 2s - 6ms/step - accuracy: 0.5338 - loss: 1.3333
========================== EDW EINAI TO MULTI333333.......
Accuracy (on the testing dataset): 53.38%
Training time:  13.289344072341919
0.32376606974088656

Accuracy (on the testing dataset): 53.38%
Training time:  13.289344072341919
0.32376606974088656

-----------------> End of second phase crossovers/mutations
                         LayerType  SamplingRate  LearningRate BatchSize  \
0   [conv, pool, conv, conv, conv]      0.378882      0.001000        64   
1  [conv, pool, conv, pool, dense]      0.378882      0.006507        64   
2   [conv, pool, conv, conv, pool]      0.378882      0.001000        47   
3   [conv, pool, conv, pool, conv]      0.378882      0.005841        64   

 

In [16]:
best_layers

[  LayerType  SamplingRate  LearningRate BatchSize Epochs  Accuracy  \
 0    [conv]      0.950000      0.001000        64      5    0.6067   
 1    [conv]      0.850000      0.001000        64      5    0.5978   
 2    [conv]      0.050000      0.008314        64      5    0.3366   
 3    [conv]      0.378882      0.001000        64      5    0.5718   
 4    [conv]      0.450000      0.001000        64      1    0.5298   
 5    [conv]      0.650000      0.001000        64      5    0.5930   
 
    TrainingSpeed     Score  
 0      24.547336  0.367982  
 1      21.621409  0.362584  
 2       6.424736  0.244120  
 3      13.650348  0.346814  
 4       7.626892  0.361823  
 5      18.740218  0.359673  ,
       LayerType  SamplingRate  LearningRate BatchSize Epochs  Accuracy  \
 0  [conv, conv]      0.650000         0.001        64      5    0.6398   
 1  [conv, conv]      0.378882         0.001        64      5    0.6307   
 2  [conv, pool]      0.313769         0.001        64      1    

In [17]:
stop_program = time.time()

# Results

In [18]:
for i in range(1, len(best_layers)):
  if i == 1:
    final_results = pd.concat([best_layers[0], best_layers[i]], axis=0)
  else:
    final_results = pd.concat([final_results, best_layers[i]], axis=0)

In [19]:
final_results

,LayerType,SamplingRate,LearningRate,BatchSize,Epochs,Accuracy,TrainingSpeed,Score
0,[conv],0.950000,0.001000,64,5,0.6067,24.547336,0.367982
1,[conv],0.850000,0.001000,64,5,0.5978,21.621409,0.362584
2,[conv],0.050000,0.008314,64,5,0.3366,6.424736,0.244120
3,[conv],0.378882,0.001000,64,5,0.5718,13.650348,0.346814
4,[conv],0.450000,0.001000,64,1,0.5298,7.626892,0.361823
5,[conv],0.650000,0.001000,64,5,0.5930,18.740218,0.359673
0,"[conv, conv]",0.650000,0.001000,64,5,0.6398,21.496531,0.388058
1,"[conv, conv]",0.378882,0.001000,64,5,0.6307,15.686642,0.382539
2,"[conv, pool]",0.313769,0.001000,64,1,0.4983,7.043961,0.350375
3,"[conv, conv]",0.470292,0.001000,64,5,0.6397,17.916364,0.387998


In [20]:
df_best = best_layers[-1].query('Score == Score.max()')
df_best

,LayerType,SamplingRate,LearningRate,BatchSize,Epochs,Accuracy,TrainingSpeed,Score
2,"[conv, pool, conv, conv, pool]",0.378882,0.001,47,5,0.7025,19.054649,0.426088


In [21]:
evolution_solution = df_best['LayerType'].tolist()
evolution_solution[0]

['conv', 'pool', 'conv', 'conv', 'pool']

In [22]:
if evolution_solution[0][-1] != 'dense':
  evolution_solution[0].append('flatten')
else:
  evolution_solution[0].insert(evolution_solution[0].index("dense"), "flatten")
evolution_solution[0].append('dense_classes_softmax')

In [23]:
print(f"Based on the Evolutionary approach the best nework is ------> {evolution_solution[0]}")

Based on the Evolutionary approach the best nework is ------> ['conv', 'pool', 'conv', 'conv', 'pool', 'flatten', 'dense_classes_softmax']


In [24]:
# Compute the time
program_time = stop_program - start_program
program_time

3085.668220281601